In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

NUM_PROGRAM = 8
programs = []
for i in range(1, NUM_PROGRAM+1):
    program = pd.read_csv('./datasets/all/Program0%d.csv' % (i))
    
    #print('Program %d' % (i))
    #print('Episodes: %d' % (len(program)))
    #print(program.columns)
    #print()
    
    #print(program.loc[:1]['Content'])
    #print()
    
    programs.append(program)


In [8]:
questions = pd.read_csv('./datasets/all/Question.csv')

print('Question')
print('Episodes: %d' % (len(questions)))
print(questions.columns)
print()

print(questions.loc[:2]['Question'])
print()

for i in range(6):
    print(questions.loc[:2]['Option%d' % (i)])
    print()

Question
Episodes: 500
Index(['Question', 'Option0', 'Option1', 'Option2', 'Option3', 'Option4',
       'Option5'],
      dtype='object')

0    媽給你送錢包來啦 來 你看一下是不是這個\n對 就是這個 你在哪裡找到它的\n
1             古人說三日不讀書 面目可憎 我覺得我最近可能臉色太難看了\n
2                         你說我們做父母的最擔心的就是這個\n
Name: Question, dtype: object

0        你看 這是我新買的錢包
1    所以想回復我昔日面貌姣好的樣子
2      我剛剛聽你媽說你要讀什麼科
Name: Option0, dtype: object

0     我的錢包不見了啦
1    是不是要定期來舉辦
2    其他老師又集體叛變
Name: Option1, dtype: object

0    以後上網咖的錢包在我身上
1         各辦理一次才對
2        聽起來好好玩天啊
Name: Option2, dtype: object

0                          什麼有錢包場
1                     能夠督促所有的用人機關
2    只是小孩自己的興趣不能得到發展 他們的心裡可能也會很悶喔
Name: Option3, dtype: object

0    早上你爸爸在車上找到的 一定是前天你放學的時候掉在車上了
1                   在上次的節目討論中也有提到
2                      走到這裡就沒有路了耶
Name: Option4, dtype: object

0         我為什麼要給你們錢包
1           超過九十分貝以上
2    每一個科目像是國語數學都很優秀
Name: Option5, dtype: object



In [13]:
import jieba

jieba.set_dictionary('./datasets/all/big5_dict.txt')

example_str = '我討厭吃蘋果'
cut_example_str = jieba.lcut(example_str)
print(cut_example_str)

def jieba_lines(lines):
    cut_lines = []
    
    for line in lines:
        cut_line = jieba.lcut(line)
        cut_lines.append(cut_line)
    
    return cut_lines

Building prefix dict from C:\Users\User\Documents\PythonWorkspace\DeepLearningLabs\datasets\all\big5_dict.txt ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.u8c4f2ae115c5d1701ba45018e0a348ee.cache
Loading model cost 0.606 seconds.
Prefix dict has been built succesfully.


['我', '討厭', '吃', '蘋果']


In [14]:
cut_programs = []

for program in programs:
    episodes = len(program)
    cut_program = []
    
    for episode in range(episodes):
        lines = program.loc[episode]['Content'].split('\n')
        cut_program.append(jieba_lines(lines))
    
    cut_programs.append(cut_program)
    
    
print("%d programs" % len(cut_programs))
print("%d episodes in program 0" % len(cut_programs[0]))
print("%d lines in first episode of program 0" % len(cut_programs[0][0]))

print()
print("first 3 lines in 1st episode of program 0: ")
print(cut_programs[0][0][:3])

8 programs
1299 episodes in program 0
635 lines in first episode of program 0

first 3 lines in 1st episode of program 0: 
[['還好', '天氣', '不錯'], ['昨天', '晚上', '的', '流星雨'], ['我', '看到', '很多', '流星']]


In [15]:
cut_questions = []
n = len(questions)

for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_lines(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)
    
    
print("%d questions" % len(cut_questions))
print(len(cut_questions[0]))

# 1 question
print(cut_questions[0][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[0][i])

500 questions
7
[['媽給', '你', '送', '錢包', '來', '啦', ' ', '來', ' ', '你', '看', '一下', '是', '不', '是', '這個'], ['對', ' ', '就是', '這個', ' ', '你', '在', '哪裡', '找到', '它', '的'], []]
['你', '看', ' ', '這是', '我', '新', '買', '的', '錢包']
['我', '的', '錢包', '不見了', '啦']
['以後', '上', '網咖', '的', '錢包', '在', '我', '身上']
['什麼', '有', '錢包', '場']
['早上', '你', '爸爸', '在', '車上', '找到', '的', ' ', '一定', '是', '前天', '你', '放學', '的', '時候', '掉', '在', '車上', '了']
['我', '為什麼', '要給', '你們', '錢包']


In [16]:
import numpy as np

np.save('cut_Programs.npy', cut_programs)
np.save('cut_Questions.npy', cut_questions)

cut_programs = np.load('cut_Programs.npy')
cut_Question = np.load('cut_Questions.npy')

In [17]:
word_dict = dict()
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1
for program in cut_programs:
    for lines in program:
        for line in lines:
            for w in line:
                add_word_dict(w)
for question in cut_questions:
    lines = question[0]
    for line in lines:
        for w in line:
            add_word_dict(w)
    
    for i in range(1, 7):
        line = question[i]
        for w in line:
            add_word_dict(w)
import operator

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print("Total %d words in word_dict" % len(word_dict))

Total 174420 words in word_dict


In [19]:
VOC_SIZE = 15000
VOC_START = 20

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
print()
print("Total %d words in voc_dict" % len(voc_dict))
np.save('voc_dict.npy', voc_dict)
voc_dict = np.load('voc_dict.npy')

[('他', 81632), ('也', 77206), ('就是', 75579), ('說', 74281), ('來', 68735), ('會', 67933), ('那', 66863), ('喔', 61530), ('可以', 60268), ('跟', 60056)]

Total 15000 words in voc_dict


In [20]:
import random

NUM_TRAIN = 10000
TRAIN_VALID_RATE = 0.7
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            program_id = random.randint(0, NUM_PROGRAM-1)
            episode_id = random.randint(0, len(cut_programs[program_id])-1)
            line_id = random.randint(0, len(cut_programs[program_id][episode_id])-2)
            
            Xs.append([cut_programs[program_id][episode_id][line_id], 
                       cut_programs[program_id][episode_id][line_id+1]])
            Ys.append(1)
            
        else:
            first_program_id = random.randint(0, NUM_PROGRAM-1)
            first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
            first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)
            
            second_program_id = random.randint(0, NUM_PROGRAM-1)
            second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
            second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)
            
            Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                       cut_programs[second_program_id][second_episode_id][second_line_id]])
            Ys.append(0)
    
    return Xs, Ys
Xs, Ys = generate_training_data()

x_train, y_train = Xs[:int(NUM_TRAIN*TRAIN_VALID_RATE)], Ys[:int(NUM_TRAIN*TRAIN_VALID_RATE)]
x_valid, y_valid = Xs[int(NUM_TRAIN*TRAIN_VALID_RATE):], Ys[int(NUM_TRAIN*TRAIN_VALID_RATE):]

In [21]:
example_doc = []

# lines in 1st episode in program 0 
for line in cut_programs[0][0]:
    example_line = ''
    for w in line:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print(example_doc[:10])

['還好 天氣 不錯 ', '昨天 晚上 ', '看到 很多 流星 ', '這次 收穫 真 豐富 ', '當然 豐富 啦 ', '說 嘛 ', '精心 製作 ', '被 一個 人 吃掉 ', '真的 嗎 ', '不要 忘記 做 秘密 檔案 ']
